In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
import time

In [2]:
nb_name = "book1.ipynb"

In [3]:
teacher_driver = webdriver.Chrome()
student_driver = webdriver.Chrome()

teacher_url = "http://localhost:9200/notebooks/" + nb_name
student_url = "http://localhost:9201/notebooks/" + nb_name

teacher_driver.get(teacher_url)
student_driver.get(student_url)

time.sleep(5)

In [19]:
def get_cell_outputs(cell):
    output_class_name = "output_subarea"
    output_elem = cell.find_elements_by_class_name(output_class_name)
    outputs = [elem.text for elem in output_elem]
    return outputs

def check_outputs_for_term(outputs, term):
    tmp = [output.find(term) > -1 for output in outputs]
    return any(tmp)

def flatten_outputs(outputs):
    tmp = ''
    for output in outputs:
        tmp += str(output)
    return tmp

In [20]:
ERR_MSGS = []

In [28]:
driver = teacher_driver

class_name = "code_cell"
# elems = driver.find_elements_by_class_name(class_name)

for i_action, cell_num in enumerate((0,1,2)):
    
    elems = driver.find_elements_by_class_name(class_name)
    cell = elems[cell_num]

    inner_class = "input_area"
    inner = cell.find_element_by_class_name(inner_class)
    inner.click()
    
    time.sleep(0.1)
    
    if cell_num == 1:        
        outputs_0 = get_cell_outputs(cell)

    action = ActionChains(driver)
    action.key_down(Keys.SHIFT)
    action.send_keys(Keys.ENTER)
    action.key_up(Keys.SHIFT)
    action.perform()    
    
    time.sleep(0.3)
                                    
    if cell_num == 1:        
        outputs_1 = get_cell_outputs(cell)
                                    
        if outputs_0 == outputs_1:
            ERR_MSGS.append("BAD JUPYTER ACTION: teacher nb test cell not executed")
                                    
    if cell_num == 2:
        outputs_2 = get_cell_outputs(cell)
        
        if not(check_outputs_for_term(outputs_2, "send_answer done.")):
            ERR_MSGS.append("BAD JUPYTER ACTION: teacher nb send_answer cell has wrong output")

In [29]:
outputs_1

['651906']

In [30]:
outputs_2

['', 'send_answer done.']

In [31]:
outputs_0

['399569']

In [9]:
driver = student_driver

class_name = "code_cell"

for i_action, cell_num in enumerate((0,1,2,3)):
    
    elems = driver.find_elements_by_class_name(class_name)
    
    cell = elems[cell_num]

    inner_class = "input_area"
    inner = cell.find_element_by_class_name(inner_class)
    inner.click()
    
    time.sleep(0.1)
    
    if cell_num == 3:
        
        action = ActionChains(driver)
        action.send_keys("nbx2.receive_answer()")
        action.perform()
        
        time.sleep(0.3)


    action = ActionChains(driver)
    action.key_down(Keys.SHIFT)
    action.send_keys(Keys.ENTER)
    
    action.key_up(Keys.SHIFT)
    action.perform()    
    
    time.sleep(0.3)